<a href="https://colab.research.google.com/github/ShachiniMekala/PoliticalCommentAnaysis/blob/main/Comments_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Background Work

In [139]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [140]:
import os
import string
import nltk
from nltk import FreqDist
import pandas as pd
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [141]:
#read corpus into dataframe
df_comments = pd.read_csv("/content/gdrive/MyDrive/NLP/Comments.csv") 

In [142]:
#select only positive comments
df_positive = df_comments[df_comments["Annotation"] == "positive"]
df_positive_comments = df_positive['Comment']

#select only negative comments
df_negative = df_comments[df_comments["Annotation"] == "negative"]
df_negative_comments = df_negative['Comment']

# Data Preprocessing

In [143]:
def preprocessing(input_str):
  output_str=''
  punc = "'"+"!"+"("+")"+"-"+"["+"]"+"{"+"}"+";"+":"+"\""+","+"<"+"+"+">"+"."+"/"+"?"+"@"+"#"+"$"+"%"+"^"+"&"+"*"+"_"+"~"+"\\"

  #remove punctuations in comment
  for char in input_str:
    if char not in punc:
        output_str = output_str + char

  output_str = output_str.strip()

  #annotate the begining and the end of the sentence
  output_str = "<" + output_str + ">"

  #convert the string into lower case and tokenize
  word_tokens = nltk.word_tokenize(output_str.lower())
  
  return word_tokens

# Corpus

In [144]:
unigram_list_p=[]
bigram_list_p=[]
unigram_list_n=[]
bigram_list_n=[]
positive_tokens_t=[]
negative_tokens_t=[]

#preprocess positive comments
for comment in df_positive_comments:
 positive_tokens = preprocessing(comment)
 #list the unigrams in the positive corpus
 unigram_list_p = unigram_list_p + list(positive_tokens)
 #list the bigrams in the positive corpus
 bigrams_p = nltk.bigrams(positive_tokens)
 bigram_list_p = bigram_list_p + list(bigrams_p)
 #concatenate all the words in positive corpus to take the unique word count
 positive_tokens_t=positive_tokens_t + positive_tokens

#preprocess negative comments
for comment in df_negative_comments:
 negative_tokens = preprocessing(comment)
 #list the unigrams in the negative corpus
 unigram_list_n = unigram_list_n + list(negative_tokens)
 #list the bigrams in the negative corpus
 bigrams_n = nltk.bigrams(negative_tokens)
 bigram_list_n = bigram_list_n + list(bigrams_n)
 #concatenate all the words in negative corpus to take the unique word count
 negative_tokens_t=negative_tokens_t+negative_tokens


In [145]:
#unique word count in corpus
unique_count = len(set(positive_tokens_t)) + len(set(negative_tokens_t))
unique_count

690

In [146]:
#frequency distribution for positive unigrams and bigrams
unigram_freq_p = FreqDist(unigram_list_p)
bigram_freq_p = FreqDist(bigram_list_p)

#frequency distribution for negative unigrams and bigrams
unigram_freq_n = FreqDist(unigram_list_n)
bigram_freq_n = FreqDist(bigram_list_n)

# Input Comment

In [147]:
#prerocess input comment
input_comment = input()

#preprocess the input comment
input_tokens=preprocessing(input_comment)
input_bigrams = list(nltk.bigrams(input_tokens))

#word count in input comment
word_count_input = len(input_tokens)

gota ta mun meka karanan den na


# Calculate Probability

In [148]:
probability_p = 1.0
# calculate the probability of being positive

for bigram in range(len(input_bigrams)):

 if input_bigrams[bigram] in bigram_freq_p:
  prob_bigram = bigram_freq_p[input_bigrams[bigram]]
 else:
  prob_bigram = 0 
 
 if input_bigrams[bigram][0] in unigram_freq_p:
  prob_unigram = unigram_freq_p[input_bigrams[bigram][0]]
 else:
  prob_unigram = 0

 #laplace smoothing
  prob_after_smoothing = (prob_bigram + 1) / (prob_unigram + unique_count)

 #probability of being positive
  probability_p = probability_p * (prob_after_smoothing)

print('Probability of being POSITIVE : '+str(probability_p))


Probability of being POSITIVE : 6.3937337358319145e-15


In [149]:
probability_n = 1.0
# calculate the probability of being negative
for bigram in range(len(input_bigrams)):

 if input_bigrams[bigram] in bigram_freq_n:
  prob_bigram = bigram_freq_n[input_bigrams[bigram]]
 else:
  prob_bigram = 0 
 
 if input_bigrams[bigram][0] in unigram_freq_n:
  prob_unigram = unigram_freq_n[input_bigrams[bigram][0]]
 else:
  prob_unigram = 0

 #laplace smoothing
  prob_after_smoothing = (prob_bigram + 1) / (prob_unigram + unique_count)

 #probability of being positive
  probability_n = probability_n * (prob_after_smoothing)

print('Probability of being NEGATIVE : '+str(probability_n))

Probability of being NEGATIVE : 4.411676277724021e-12


# Classification

In [150]:
if probability_p > probability_n:
 print("Input is a POSITIVE comment")
 perplexity=pow(probability_p, (-1 / word_count_input))
 print("Perplexity : "+str(perplexity))

else:
 if probability_p < probability_n:
  print("Input is a NEGATIVE comment")
  perplexity=pow(probability_n, (-1 / word_count_input))
  print("Perplexity : "+str(perplexity))
 else:
  print("Failed to Classify")

Input is a NEGATIVE comment
Perplexity : 18.26882884295643
